In [12]:
from openai import OpenAI
#create a folder for the audio
import os
import time
from config import client_secret
import os
from dotenv import load_dotenv
# Load .env file
load_dotenv('../.env')


# get secert key from .env
client_secret = os.getenv("OPENAI_SECERT")

client = OpenAI(
    api_key=client_secret
)


ModuleNotFoundError: No module named 'config'

In [9]:
# firebase
from firebase_admin import credentials, firestore, initialize_app
from firebase_admin import storage
cred = credentials.Certificate("../lang-learning-app-gpt-firebase-adminsdk-h6lgl-379786af55.json")

app = initialize_app(cred, {  "storageBucket": "lang-learning-app-gpt.appspot.com"
})
db = firestore.client(app=app)
bucket = storage.bucket(app=app)
# initialize_app(cred, {  "storageBucket": "lang-learning-app-gpt.appspot.com"
# }, app)
print("firebase connected")



ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

Get audio for the words

In [10]:
import json
json_file = 'beginner_lesson.json'
data = {}
# get the json file
with open(json_file) as f:
    data = json.load(f)
    print(data)

exercise = data['exercises']
print(len(exercise))



# get the list of lessons
question_text = []
for lesson in exercise:
    if isinstance(lesson, dict) :
        if lesson['taskType'] == 'Speaking' or lesson['taskType'] == 'Reading':
            question_text.append(lesson['text'])

print(question_text)


{'lessonTitle': 'French Beginner - First Lesson', 'skillLevel': 'Beginner', 'focusArea': 'Basic Greetings and Common Phrases', 'totalTasks': 35, 'exercises': [{'taskType': 'Speaking', 'taskDescription': "Repeat the greeting 'Bonjour' (Hello)", 'text': 'Bonjour', 'translation': 'Hello', 'audioFilePath': 'https://storage.googleapis.com/lang-learning-app-gpt.appspot.com/lessons/French%20Beginner%20-%20First%20Lesson/Bonjour.mp3/Bonjour.mp3', 'points': 5}, {'taskType': 'Speaking', 'taskDescription': "Repeat 'Comment ça va?' (How are you?)", 'text': 'Comment ça va?', 'translation': 'How are you?', 'audioFilePath': 'https://storage.googleapis.com/lang-learning-app-gpt.appspot.com/lessons/French%20Beginner%20-%20First%20Lesson/Comment%20%C3%A7a%20va%3F.mp3/Comment%20%C3%A7a%20va%3F.mp3', 'points': 5}, {'taskType': 'Speaking', 'taskDescription': "Say 'Je m'appelle John' (My name is John)", 'text': "Je m'appelle John", 'translation': 'My name is John', 'audioFilePath': 'https://storage.googleap

In [51]:
#create the audio from the text using openai text to speech


if not os.path.exists('lesson_audio'):
    os.makedirs('lesson_audio')

list_of_files = os.listdir('lesson_audio')

#if the file is already there, skip it




for text in question_text:
    if f'{text}.mp3' in list_of_files:
        print(f'{text}.mp3 already exists')
        continue


    try:
        response = client.audio.speech.create(
        model="tts-1-hd",
        voice="alloy",
        input=text

        )
        response.stream_to_file(f'lesson_audio/{text}.mp3')

    except Exception as e:
        print(e)
        time.sleep(21)
        response = client.audio.speech.create(
        model="tts-1-hd",
        voice="shimmer",
        input=text

        )
        response.stream_to_file(f'lesson_audio/{text}.mp3')
        continue



Bonjour.mp3 already exists
Comment ça va?.mp3 already exists
Je m'appelle John.mp3 already exists
Merci.mp3 already exists
S'il vous plaît.mp3 already exists
Au revoir.mp3 already exists
Je suis désolé(e).mp3 already exists
Pouvez-vous m'aider?.mp3 already exists
Error code: 429 - {'error': {'message': 'Rate limit reached for tts-1-hd in organization org-xN42YthJ7m3elLOthSqIkZtP on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Error code: 429 - {'error': {'message': 'Rate limit reached for tts-1-hd in organization org-xN42YthJ7m3elLOthSqIkZtP on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Error code: 429 - {'error': {'message': 'Rate limit reached 

fix the audio

In [11]:
text = "Au revoir"

response = client.audio.speech.create(
        model="tts-1-hd",
        voice="alloy",
        input=text

        )
response.stream_to_file(f'lesson_audio/{text}.mp3')


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-Qf6aF***************************************cJLJ. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

add all to firebase storage

In [6]:
# upload the audio to firebase
from firebase_admin import storage
import datetime
import os


list_of_files = os.listdir('lesson_audio')
print(len(list_of_files), 'files in lesson_audio')

bucket = storage.bucket()
list_of_files = os.listdir('lesson_audio')

for file in list_of_files:
    folder_path = f'lessons/{data["lessonTitle"]}/{file}'  # Specify the folder path here

    if file.endswith('.mp3'):
        blob = bucket.blob(f'{folder_path}/{file}')  # Specify the folder path as part of the blob name
        # upload location
        upload_location = f'lesson_audio/{file}'

        blob.upload_from_filename(upload_location)  # upload the file
        print(f'uploaded {file} to firebase')
        blob.make_public()

        # get the url of the file
        url = blob.public_url
        print(url)
        for lesson in exercise:
            # update the lesson with the url at audioFilePath
            if isinstance(lesson, dict):
                if lesson['text'] == file[:-4]:
                    lesson['audioFilePath'] = url  # add the url to the lesson
                    print(lesson)


#update the json file
with open(json_file, 'w') as f:
    json.dump(data, f, indent=4)





28 files in lesson_audio
uploaded Elle parle anglais et français.mp3 to firebase
https://storage.googleapis.com/lang-learning-app-gpt.appspot.com/lessons/French%20Beginner%20-%20First%20Lesson/Elle%20parle%20anglais%20et%20fran%C3%A7ais.mp3/Elle%20parle%20anglais%20et%20fran%C3%A7ais.mp3
{'taskType': 'Reading', 'taskDescription': "Translate 'Elle parle anglais et français' into English", 'text': 'Elle parle anglais et français', 'translation': 'She speaks English and French', 'audioFilePath': 'https://storage.googleapis.com/lang-learning-app-gpt.appspot.com/lessons/French%20Beginner%20-%20First%20Lesson/Elle%20parle%20anglais%20et%20fran%C3%A7ais.mp3/Elle%20parle%20anglais%20et%20fran%C3%A7ais.mp3', 'points': 7}
uploaded Voir vous plus tard.mp3 to firebase
https://storage.googleapis.com/lang-learning-app-gpt.appspot.com/lessons/French%20Beginner%20-%20First%20Lesson/Voir%20vous%20plus%20tard.mp3/Voir%20vous%20plus%20tard.mp3
{'taskType': 'Writing', 'taskDescription': "Write 'See you la

In [7]:
# reget all the data and add it to firebase
# get the json file
with open(json_file) as f:
    data = json.load(f)
    print(data)

#update the lesson in firebase
doc_ref = db.collection(u'lessons').document(data['lessonTitle'])
doc_ref.set(data)
print(f'updated {data["lessonTitle"]} in firebase')


{'lessonTitle': 'French Beginner - First Lesson', 'skillLevel': 'Beginner', 'focusArea': 'Basic Greetings and Common Phrases', 'totalTasks': 35, 'exercises': [{'taskType': 'Speaking', 'taskDescription': "Repeat the greeting 'Bonjour' (Hello)", 'text': 'Bonjour', 'translation': 'Hello', 'audioFilePath': 'https://storage.googleapis.com/lang-learning-app-gpt.appspot.com/lessons/French%20Beginner%20-%20First%20Lesson/Bonjour.mp3/Bonjour.mp3', 'points': 5}, {'taskType': 'Speaking', 'taskDescription': "Repeat 'Comment ça va?' (How are you?)", 'text': 'Comment ça va?', 'translation': 'How are you?', 'audioFilePath': 'https://storage.googleapis.com/lang-learning-app-gpt.appspot.com/lessons/French%20Beginner%20-%20First%20Lesson/Comment%20%C3%A7a%20va%3F.mp3/Comment%20%C3%A7a%20va%3F.mp3', 'points': 5}, {'taskType': 'Speaking', 'taskDescription': "Say 'Je m'appelle John' (My name is John)", 'text': "Je m'appelle John", 'translation': 'My name is John', 'audioFilePath': 'https://storage.googleap